# Error computation with HESSE and MINOS

This is a topical tutorial. If you are new to iminuit, you should go through the basic tutorial first. 

iminuit (and C++ MINUIT) offers two major ways of computing parameter uncertainties from the cost function (which must be of least-squares form or negative log-likelihood), HESSE and MINOS. These have different pros and cons, on which we already touched in the basic tutorial. Here we want to go a bit deeper into the pros and cons and also try to reveal a bit of the math behind the two approaches.

There are several sources which explain what HESSE and MINOS, in particular the MINUIT User's Guide (linked from the [iminuit documentation](https://iminuit.readthedocs.io/en/latest/about.html)). The mathematical details are covered in F. James, "Statistical Methods in Experimental Physics", 2nd edition, World Scientific (2006).

## Definitions

We need to use a couple of technical terms from mathematical statistics when talk about proven properties. We also use recurring variable names with the same meaning throughout.

* $n$ number of data samples
* $x$ data value from data distribution $f(x; \theta)$ with parameter $\theta$ (not an angle!)
* $\hat \theta$ estimate of $\theta$ obtained from a sample (the parameter value obtained from the fit)
* $\mathcal L$ likelihood function
* $\mathcal I$ [Fisher information](https://en.wikipedia.org/wiki/Fisher_information)
* $E[\dots]$ expectation value of $\dots$ over distribution of $x$
* $V[\dots]$ variance of $\dots$

The terms **asymptotic** and **asymptotic limit** refer to inference from an infinite data sample. Mathematical properties of statistical methods are most commonly computed in this limit.

## HESSE

HESSE in a nutshell:
* Computes approximate covariance matrix $C_{ij}$ for all fitted parameters
* Square-root of diagonal elements of $C_{ij}$ corresponds to one standard deviation; to obtain $k$ standard deviations, multiply covariance matrix $C$ by $k^2$
* Can be used to form confidence intervals $\hat \theta_i \pm C_{ii}^{1/2}$
* Only way to obtain parameter correlations
* Comparably fast, requires:
    * Numerical computation of all second derivatives of the cost function (Hessian matrix)
    * Inverting this matrix
* Cannot compute uncertainty for only one parameter if there are several
* Approximately computed as by-product of MIGRAD algorithm (usually accurate enough when strategy >= 1 is used)

The algorithm derives its name from the Hessian matrix of second derivatives which is computed to obtain the uncertainties.

### How does it work?

The matrix of second derivatives of the cost function is computed for all free parameters and multiplied by 2. The result is inverted and multiplied by `errordef`.

MINUIT implements parameter limits by applying a variable transformation and therefore distinguishes internal and external parameter space. The Hessian matrix is computed in the internal parameter space and transformed using the chain rule to external space.

### Why does it work?

One can prove under general conditions in the asymptotic limit that a parameter estimate $\hat\theta$ obtained from the least-squares and the maximum-likelihood methods is normally distributed with minimum variance, given by the [Cramer-Rao lower bound](https://en.wikipedia.org/wiki/Cram%C3%A9r%E2%80%93Rao_bound), which is the minimum for any unbiased estimator (these methods are asymptotically unbiased).

$$
V(\hat \theta) \underset{n\rightarrow \infty}{\longrightarrow} \left\{ n E\left[\left( \frac{\partial \ln\! f(x;\theta)}{\partial \theta} \right)^2 \right]_{\theta = \hat\theta} \right\}^{-1} = \left\{ -n E\left[\frac{\partial^2 \ln\! f(x;\theta)}{\partial \theta^2} \right]_{\theta = \hat\theta} \right\}^{-1}
$$

The expectation here is taken over the data distribution. Since the expectation value is constant, we see that the variance of $\hat\theta$ scales goes down with $n^{-1}$ in the asymptotic limit.

If the data range is independent of $\theta$, which we usually assume (but see F. James book for a counter example), we can swap integration over $x$ and differentiation with respect to $\theta$. Doing this and replacing the expectation with its plug-in estimate, the arithmetic average, we obtain:

$$
-n E\left[\frac{\partial^2 \ln\! f(x;\theta)}{\partial \theta^2} \right]_{\theta = \hat \theta} = -n \frac{1}{n} \sum_i \frac{\partial^2 \ln\! f(x_i; \theta)}{\partial \theta^2}\Big\vert_{\theta = \hat \theta} = \frac{\partial^2 \big(-\sum_i \ln\! f(x_i; \theta)\big)}{\partial \theta^2}\Big\vert_{\theta = \hat \theta}
$$

We now see that the numerator contains the negative log-likelihood  function that we often plug into iminuit. If there is a vector of parameters $\hat{\vec \theta}$, then this turns into the Hessian matrix of second derivatives.

#### A bit of history

So what about these factors of $2$ and `errordef` in MINUIT that were mentioned above? These don't appear here. There are historical reasons for those. In the asymptotic limit, the least-squares cost function that corresponds to the log-likelihood is $Q = -2 \ln\! \mathcal{L} - \text{constants}$. MINUIT was originally developed with least-squares fits in mind, therefore its internal math assumes the $Q$ form. If the second derivatives are computed from $Q$, the constants are removed but the Hessian must be multiplied by a factor of two to get the right variance. Correspondingly, if the user puts in a negative log-likelihood function, the same procedure now introduces an extra factor of two $2$, which must be compensated by the `errordef` value of 0.5 for the negative log-likelihood.

### Why is HESSE approximate

We started out from Cramer-Rao bound, the asymptotic limit for the parameter variance. How fast the finite sample approaches the limit depends on the problem at hand. For normal distributed data, the bound is exact.

We further approximated the computation of the bound by replacing the expectation over the likelihood with the sample mean of the likelihood. 

## MINOS

MINOS in a nutshell:

* Approximate confidence intervals (intervals are wrongly claimed to be "exact" in some sources, including the MINUIT paper from 1975)
* Cannot compute parameter correlations
* Some (unverified) sources claim better coverage probability than intervals based on HESSE
* In general slower than HESSE (requiring more function evaluations):
    * Iteratively computes the value pair $\theta^d, \theta^u$ which increases the cost function by `errordef` over the minimum
    * If the cost function has several parameters, it is minimised with respect to all other parameters during this scan
* Can be used to compute uncertainty for only one parameter - but this is not more efficient than HESSE, since the computation requires at least one evaluation of HESSE

The MINOS algorithm computes [profile-likelihood based](https://en.wikipedia.org/wiki/Likelihood_function#Profile_likelihood) confidence intervals.

### How does it work?

MINOS scans the likelihood along one parameter $\theta_i$,  while minimizing the likelihood with respect to all other parameters $\theta_k$ with $k \ne i$. This is effectively the same as expressing the other parameter estimates as a function of $\theta_i$, $\hat \theta_k(\theta_i)$, and scanning the now one-dimensional negative log-likelihood $-\ln \mathcal{L}(\theta_i; \theta_k = \hat \theta_k(\theta_i) , k\ne i)$. This is called the [profile likelihood](https://en.wikipedia.org/wiki/Likelihood_function#Profile_likelihood) for parameter $\theta_i$.

One follows this curve until $-\ln \mathcal{L}$ increases by `errordef` with respect to its minimum and stores the two corresponding values $\theta^d_i$ and $\theta^u_i$. The interval $(\theta^d_i, \theta^u_i)$ has $68\,\%$ coverage probability in the asymptotic limit. In multi-dimensional parameter space, the computation is comparably expensive due to the iterative steps of scanning and minimization.

An efficient algorithm to compute the interval is described by Venzon and Moolgavkar in *A Method for Computing Profile-Likelihood-Based Confidence Intervals*, Journal of the Royal Statistical Society C37 (1988) 87–94 [DOI](https://doi.org/10.2307%2F2347496).

### Why does it work?

We define the likelihood ratio $\ell(\vec\theta) = \mathcal{L}(\vec\theta) / \mathcal{L}(\hat{\vec\theta})$. In the asymptotic limit, $-2 \ln \ell(\vec\theta)$ is $\chi^2(k)$ distributed, where $k$ is the number of fitted parameters. For $k = 1$, the $\chi^2$-interval $[0, 1)$ contains $68\,\%$ probability. For a single parameter fit therefore a corresponding interval is found by the two values $\theta^d$ and $\theta^u$ which solve

$$
-2\ln\ell(\theta) = 1 \Leftrightarrow -\ln\ell(\theta) = 1/2
$$

We recognize the difference in the negative log-likelihood on the left-hand side and our `errordef = 0.5` on the right-hand side. Confidence intervals with other coverage probability can be constructed by finding the corresponding upper value of the $\chi^2$-interval with that integrated probability. In a least-squares fit, we have

$$
\Delta Q(\theta) = -2 \ln \ell(\theta) = 1
$$

therefore `errordef = 1` is the crossing value for a least-squares cost function.

In the multi-parameter case, when we search for the interval for $\theta_i$, the other parameters are effectively fixed to their best-fit values for the current value $\theta_i$ in the scan, $\theta_k = \hat\theta_k(\theta_i)$. Therefore, during the scan they are not free. The profile likelihood is effectively a likelihood for a single parameter. Thus, $68\,\%$-intervals are also here obtained when the negative log-likelihood crosses the value $1/2$ above the minimum. 

### Why is MINOS approximate

In some sources, the MINOS intervals are wrongly described as *exact*. They are not exact, because for finite samples the intervals do not necessarily have $68\,\%$ coverage probability, only in the asymptotic limit.

One can argue that two approximations are involved in the HESSE calculation of an interval, but only one in the MINOS calculation. The MINOS intervals therefore could approach the asymptotic limit more rapidly. Seeing practical examples which demonstrate this would be very interesting.